In [1]:
import pandas as pd
from catboost import CatBoostClassifier

from sklearn.metrics import roc_auc_score, accuracy_score
from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
model_name = "cat_02"

## Load data

In [ ]:
train_data = pd.read_csv("processed_dataset/train.csv", index_col=0)
validate_data = pd.read_csv("processed_dataset/validation.csv", index_col=0)
test_data = pd.read_csv("processed_dataset/test.csv", index_col=0)

train_target = pd.read_csv("processed_dataset/target_train.csv", index_col=0)
validate_target = pd.read_csv("processed_dataset/target_validate.csv", index_col=0)

In [ ]:
validate_target = pd.read_csv("processed_dataset/target_validate.csv", index_col=0)["radiant_win"].astype("int")

In [ ]:
train_target = train_target["radiant_win"].astype("int")
validate_target = validate_target["radiant_win"].astype("int")

In [ ]:
train_data = train_data.drop(["match_id_hash", "game_time"], axis=1)
validate_data = validate_data.drop(["match_id_hash", "game_time"], axis=1)
test_data = test_data.drop(["match_id_hash", "game_time"], axis=1)

In [ ]:
train_data.shape

## Create model

In [ ]:
clf = CatBoostClassifier(n_estimators=700,
                         max_depth = 5)

In [ ]:
clf.fit(train_data, train_target, verbose=True)

In [ ]:
validate_target.shape

In [ ]:
validate_predict = clf.predict_proba(validate_data)[:,1]
test_predict = clf.predict_proba(test_data)[:,1]

print("MODEL SCORING")
print("AUC:", roc_auc_score(validate_target, validate_predict))
print("ACC:", accuracy_score(validate_target, validate_predict.round()))

AUC: 0.8216585820565936
ACC: 0.740679012345679

In [ ]:
test_ids = pd.read_csv("processed_dataset/test.csv", index_col=0)["match_id_hash"]

In [ ]:
submit_data = {
    "match_id_hash": test_ids,
    "radiant_win_prob": test_predict
}
submit = pd.DataFrame(submit_data)
submit.to_csv("test_predict/%s.csv" % model_name, index=False)


submit_data = {
    "radiant_win_prob": validate_predict
}

submit = pd.DataFrame(submit_data)
submit.to_csv("validate_predict/%s.csv" % model_name, index=False)

In [ ]:
sorted(list(zip(clf.feature_importances_, train_data.columns)))[::-1]